**Семинар 2.**

Пусть у нас имеется некоторый набор данных sales.csv о продажах квартир.

— цена продажи (SalePrice, зависимая переменная),

— наземная жилая зона (GrLivArea),

— общее качество (Overall Quality),

— территория гаража (GarageArea),

— условия сделки (SaleCondition).

Реализуйте построение модели линейной регрессии на этих данных. Проверьте качество модели на обучающей и тестовой выборках с помощью MAE, MSE.

In [23]:
import pandas as pd
df = pd.read_csv('housing.csv')
# Display the first few rows of the DataFrame
print(df.head())

   longitude  latitude  ...  median_house_value  ocean_proximity
0    -122.23     37.88  ...            452600.0         NEAR BAY
1    -122.22     37.86  ...            358500.0         NEAR BAY
2    -122.24     37.85  ...            352100.0         NEAR BAY
3    -122.25     37.85  ...            341300.0         NEAR BAY
4    -122.25     37.85  ...            342200.0         NEAR BAY

[5 rows x 10 columns]


Добавим функции из предыдущего семинара, для оценки MSE и MSA

In [24]:
def calculate_mse(y_true, y_pred):
    n = len(y_true)
    mse = sum((y_true[i] - y_pred[i]) ** 2 for i in range(n)) / n

    return mse


def calculate_mae(y_true, y_pred):
    n = len(y_true)
    mae = sum(abs(y_true[i] - y_pred[i]) for i in range(n)) / n

    return mae

Реализуем метод наименьших квадратов

In [25]:
def least_squares(x, y):
    #Вычисляеv параметры линейной модели y = mx + b с использованием метода наименьших квадратов
    n = len(x)
    if n != len(y):
        raise ValueError("Длины списков x и y должны совпадать.")

    # Вычисляем необходимые суммы
    sum_x = sum(x)
    sum_y = sum(y)
    sum_xy = sum(x[i] * y[i] for i in range(n))
    sum_x_squared = sum(xi ** 2 for xi in x)
    # Вычисляем наклон (m) и пересечение (b)
    m = (n * sum_xy - sum_x * sum_y) / (n * sum_x_squared - sum_x ** 2)
    b = (sum_y - m * sum_x) / n
    return m, b

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [27]:
zero_count = (df['total_bedrooms'] == 0).sum()
print(zero_count)

0


так как помещений с нулём ванных комнат нет в датафрейме, то заменим пустые значения на нуль

In [28]:
df['total_bedrooms'] = df['total_bedrooms'].fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20640 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


Проверим наличие дубликатов

In [29]:
duplicate_rows_data = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_data.shape)

number of duplicate rows:  (0, 10)


Дубликатов нет

Построим модель и обучим её


In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
X = df.drop(["median_house_value","ocean_proximity"], axis=1)
y = df["median_house_value"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
lr = LinearRegression()
lr.fit(X_train, y_train)

# Предсказания для обучающей выборки
y_train_pred = lr.predict(X_train)

# Предсказания для тестовой выборки
y_test_pred = lr.predict(X_test)

# Приведение y_train и y_train_pred к одному индексу
y_train = y_train.reset_index(drop=True)
y_train_pred = pd.Series(y_train_pred)

# Вычисление ошибок для обучающей выборки
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Приведение y_test и y_test_pred к одному индексу
y_test = y_test.reset_index(drop=True)
y_test_pred = pd.Series(y_test_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f'Обучающая выборка:\nMSE: {mse_train}\nMAE: {mae_train}\n')
print(f'Тестовая выборка:\nMSE: {mse_test}\nMAE: {mae_test}')

Обучающая выборка:
MSE: 4811134397.884198
MAE: 50626.79448983797

Тестовая выборка:
MSE: 5248005521.732874
MAE: 52615.05867896075


А что если попробовать построить модель на основе зависимости лишь от двух атрибутов:"total_rooms", "median_income"?

In [33]:
X = df[["total_rooms", "median_income"]]
y = df["median_house_value"]
# Разделение данных на обучающую и тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
lr = LinearRegression()
lr.fit(X_train, y_train)

# Предсказания для обучающей выборки
y_train_pred = lr.predict(X_train)

# Предсказания для тестовой выборки
y_test_pred = lr.predict(X_test)

# Приведение y_train и y_train_pred к одному индексу
y_train = y_train.reset_index(drop=True)
y_train_pred = pd.Series(y_train_pred)

# Вычисление ошибок для обучающей выборки
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Приведение y_test и y_test_pred к одному индексу
y_test = y_test.reset_index(drop=True)
y_test_pred = pd.Series(y_test_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f'Обучающая выборка:\nMSE: {mse_train}\nMAE: {mae_train}\n')
print(f'Тестовая выборка:\nMSE: {mse_test}\nMAE: {mae_test}')

Обучающая выборка:
MSE: 6991326571.517395
MAE: 62490.70552650054

Тестовая выборка:
MSE: 7091376214.422123
MAE: 62987.5275786113


Стало хуже, но на полном дата сете возникает переобучение.
Мне сложно сказать насколько хороши модели  в целом, но при сравнении моделей друг с другом метрики показывают насколько модели хороши относительно друг друга